<a href="https://colab.research.google.com/github/brc7ichikawa/google_colab/blob/master/keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import numpy as np 
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from make_tensorboard import make_tensorboard
import sys

np.random.seed(1671)
NB_EPOCH = 20
BATCH_SIZE = 128
VARBOSE = 1
NB_CLASSES = 10 
OPTIMIZER = Adam()
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2
DROPOUT = 0.3
MODEL_DIR = './tmp'

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
RESHAPED = 784 

X_train = X_train.reshape(60000,RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

Y_train = np_utils.to_categorical(Y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(Y_test, NB_CLASSES)

model = Sequential()
model.add(Dense(N_HIDDEN, input_shape = (RESHAPED,)))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(N_HIDDEN))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(NB_CLASS))
model.add(Activation('softmax'))
model.summary()

model.compile(loss= 'categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

#callbacks = [make_tensorboard(set_dir_name = 'keras_MINST_V3')]
callbacks = [ModelCheckpoint(save_best_only = True, filepath = os.path.join(MODEL_DIR, "model-{epoch:02d}.h5"))]
model.fit(X_train,
         Y_train,
         batch_size = BATCH_SIZE, epochs = NB_EPOCH,
         callbacks = callbacks,
         verbose = VARBOSE,
         validation_split=VALIDATION_SPLIT)
score = model.evaluate(X_test, Y_test, verbose=VARBOSE)
print('TEST SCORE:', score[0])
print('TEST accuracy:', score[1])


X_train: (60000, 28, 28)
X_train_reshape: (60000, 784)


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
!ls -alh

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
import os, sys
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten, Dropout
from keras.layers.core import Dense
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

def lenet(input_shape, num_classes):
  model = Sequential()
  model.add(Conv2D(20, kernel_size = 5, padding = 'same',
                   input_shape = input_shape, activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (2, 2)))
  model.add(Conv2D(50, kernel_size = 5, padding = 'same', activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (2, 2)))
  model.add(Flatten())
  model.add(Dense(500, activation = 'relu'))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))
  return model
  
class MNISTDataSet():
  
  def __init__(self):
    self.image_shape = (28,28,1)
    self.num_classes = 10
    
  def get_batch(self):
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = [self.processes(d) for d in[x_train, x_test]]
    y_train, y_test = [self.processes(d, label_data = True) for d in [y_train, y_test]]
    return x_train, y_train, x_test, y_test
  
  def processes(self, data, label_data = False):
    if label_data:
      data = keras.utils.to_categorical(data, self.num_classes)
      #print(data.shape)
    else:
      data = data.astype('float32')
      data /= 255
      
      shape = (data.shape[0],) + self.image_shape
      #print(shape)
      
      data = data.reshape(shape)
      #print(data[0][20])
      #sys.exit()
    return data

class Trainer():
  def __init__(self, model, loss, optimizer):
    self._target = model
    self._target.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])
    self.verbose = 1
    self.log_dir = os.path.join(os.path.dirname("make_tensorboard.py"), 'logdir')
    
  def train(self, x_train, y_train, batch_size, epochs, validation_split):
    if os.path.exists(self.log_dir):
      import shutil
      shutil.rmtree(self.log_dir)
    os.mkdir(self.log_dir)
    
    self._target.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, 
                    validation_split = validation_split, callbacks=[TensorBoard(log_dir = self.log_dir)],
                    verbose = self.verbose)
    
mnist_ = MNISTDataSet()
x_train, y_train, x_test, y_test = mnist_.get_batch()
model = lenet(mnist_.image_shape, mnist_.num_classes)
trainer = Trainer(model, loss = "categorical_crossentropy", optimizer=Adam())
trainer.train(x_train, y_train, batch_size = 128, epochs = 12, validation_split = 0.2)

score = model.evaluate(x_test, y_test, verbose = 0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [0]:
!tensorboard --logdir=./logdir

TensorBoard 1.12.2 at http://fcb6bd4f03c0:6006 (Press CTRL+C to quit)
^C
